# 1. Write_data_to_S3

#### Tabular_dataset을 위해서는 [Amazon Customer Reviews Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html) 을 사용하여 작업합니다.

Amazon Customer reviews는 1995 년 첫 번째 review 이후 20 년이 넘는 기간 동안 수백만 명의 아마존 고객이 1 억 건 이상의 리뷰를 통해 Amazon.com 웹 사이트의 제품에 대한 의견을 표명하고 자신의 경험을 설명했습니다. 이로 인해 Amazon Customer Reviews는 자연어 처리 (NLP), 정보 검색 (IR) 및 기계 학습 (ML) 분야의 학술 연구원을위한 풍부한 정보 소스가됩니다. 

dataset을 Amazon S3 bucket에 Upload를 하게 됩니다. 데이터셋은 2가지 포맷을 모두 이용할 수 있습니다.

* Tab separated value (TSV), a text format - `s3://amazon-reviews-pds/tsv/`
* Parquet, an optimized columnar binary format - `s3://amazon-reviews-pds/parquet/`

Parquet dataset은 쿼리 성능을 개선하기 위해 column에 의해 파티션되어 있습니다. 즉, subfolder들로 구분되어 있습니다.카테고리에 특정한 데이터를 읽고자 할 경우에는 SQL queries에서 product_category에 대해 `WHERE`을 사용할 수 있습니다.

이 작업은 the AWS Command Line Interface (CLI)를 이용하여 S3의 bucket 내용을 확인할 수 있습니다.


#### Dataset columns

- `marketplace`: review가 작성된 마켓 플레이스의 2자리 국가 코드 ("US").
- `customer_id`: review 집계에 사용되는 임의의 작성자 식별 값
- `review_id`: review의 고유 ID
- `product_id`: reviews와 관련된 제품의 고유 ID (ASIN).  `http://www.amazon.com/dp/<ASIN>` 에서 제품의 상세 정보를 볼 수 있음
- `product_parent`: 동일 제품에 대해 review를 집계하는데 사용하는 임의의 식별자
- `product_title`: 제품명
- `product_category`: review를 그룹화할 수 있는 제품 카테고리 (digital videos).
- `star_rating`: review에 대한 1-5 등급
- `helpful_votes`:유용한 review의 투표 수
- `total_votes`: review가 받은 전체 투표 수
- `vine`: [Vine](https://www.amazon.com/gp/vine/help) 프로그램(reviewer 프로그램)에 의해 적힌 review 여부
- `verified_purchase`: 확인 구매에 대한 review 여부
- `review_headline`: review 제목
- `review_body`: review 내용
- `review_date`: review가 적힌 날짜





In [1]:
%store -r

# Set S3 Source Location

In [2]:
s3_source_path_tsv = 's3://amazon-reviews-pds/tsv/'

In [3]:
!aws s3 ls $s3_source_path_tsv

2017-11-24 13:22:50          0 
2017-11-24 13:48:03  241896005 amazon_reviews_multilingual_DE_v1_00.tsv.gz
2017-11-24 13:48:17   70583516 amazon_reviews_multilingual_FR_v1_00.tsv.gz
2017-11-24 13:48:34   94688992 amazon_reviews_multilingual_JP_v1_00.tsv.gz
2017-11-24 13:49:14  349370868 amazon_reviews_multilingual_UK_v1_00.tsv.gz
2017-11-24 13:48:47 1466965039 amazon_reviews_multilingual_US_v1_00.tsv.gz
2017-11-24 13:49:53  648641286 amazon_reviews_us_Apparel_v1_00.tsv.gz
2017-11-24 13:56:36  582145299 amazon_reviews_us_Automotive_v1_00.tsv.gz
2017-11-24 14:04:02  357392893 amazon_reviews_us_Baby_v1_00.tsv.gz
2017-11-24 14:08:11  914070021 amazon_reviews_us_Beauty_v1_00.tsv.gz
2017-11-24 14:17:41 2740337188 amazon_reviews_us_Books_v1_00.tsv.gz
2017-11-24 14:45:50 2692708591 amazon_reviews_us_Books_v1_01.tsv.gz
2017-11-24 15:10:21 1329539135 amazon_reviews_us_Books_v1_02.tsv.gz
2017-11-24 15:22:13  442653086 amazon_reviews_us_Camera_v1_00.tsv.gz
2017-11-24 15:27:13 2689739299 amazon_rev

# Set S3 Destination Location

In [4]:
s3_destination_path_tsv = 's3://{}/amazon-reviews-pds/tsv/'.format(data_bucket)
print(s3_destination_path_tsv)

s3://sagemaker-us-east-1-322537213286/amazon-reviews-pds/tsv/


In [5]:
!aws s3 cp --recursive $s3_source_path_tsv $s3_destination_path_tsv --exclude "*" --include "amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
!aws s3 cp --recursive $s3_source_path_tsv $s3_destination_path_tsv --exclude "*" --include "amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
!aws s3 cp --recursive $s3_source_path_tsv $s3_destination_path_tsv --exclude "*" --include "amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"

copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz to s3://sagemaker-us-east-1-322537213286/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz to s3://sagemaker-us-east-1-322537213286/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz
copy: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz to s3://sagemaker-us-east-1-322537213286/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz


#### List files

In [6]:
!aws s3 ls $s3_destination_path_tsv

2020-09-23 12:58:31   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-09-23 12:58:33   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz
2020-09-23 12:58:38  193389086 amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz


In [7]:
%store s3_destination_path_tsv

Stored 's3_destination_path_tsv' (str)
